# Federal CTC Expansion Reduces MA State Tax

This notebook demonstrates how federal CTC expansion reduces Massachusetts state tax for household 4428 (tax unit 442801).

In [1]:
from policyengine_us import Microsimulation
from policyengine_us.model_api import *
import numpy as np

DATASET = "hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
YEAR = 2026
TAX_UNIT_ID = 442801

## Step 1: Without CTC Expansion

In [2]:
# Reform stack WITHOUT CTC
reform_no_ctc = Reform.from_dict({
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.income.bracket.rates.4": {"2026-01-01": 0.28},
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
}, country_id="us")

sim_no_ctc = Microsimulation(reform=reform_no_ctc, dataset=DATASET)

# Find our tax unit
tax_unit_ids = sim_no_ctc.calculate("tax_unit_id", period=YEAR).values
tu_idx = np.where(tax_unit_ids == TAX_UNIT_ID)[0][0]

itemizes_no_ctc = bool(sim_no_ctc.calculate('tax_unit_itemizes', YEAR).values[tu_idx])
ma_tax_no_ctc = sim_no_ctc.calculate('ma_income_tax', YEAR).values[tu_idx]
ma_exemption_no_ctc = sim_no_ctc.calculate('ma_part_b_taxable_income_exemption', YEAR).values[tu_idx]

print("WITHOUT CTC EXPANSION:")
print(f"  Federal itemizes: {itemizes_no_ctc}")
print(f"  MA state tax: ${ma_tax_no_ctc:,.2f}")
print(f"  MA Part B exemption: ${ma_exemption_no_ctc:,.2f}")

WITHOUT CTC EXPANSION:
  Federal itemizes: False
  MA state tax: $-4,409.01
  MA Part B exemption: $12,800.00


## Step 2: With CTC Expansion

In [3]:
# Add CTC expansion
reform_with_ctc = Reform.from_dict({
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.income.bracket.rates.4": {"2026-01-01": 0.28},
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
    # CTC expansion parameters
    "gov.contrib.reconciliation.ctc.in_effect": {"2026-01-01": True},
    "gov.irs.credits.ctc.amount.base[0].amount": {"2026-01-01": 2200},
}, country_id="us")

sim_with_ctc = Microsimulation(reform=reform_with_ctc, dataset=DATASET)

itemizes_with_ctc = bool(sim_with_ctc.calculate('tax_unit_itemizes', YEAR).values[tu_idx])
ma_tax_with_ctc = sim_with_ctc.calculate('ma_income_tax', YEAR).values[tu_idx]
ma_exemption_with_ctc = sim_with_ctc.calculate('ma_part_b_taxable_income_exemption', YEAR).values[tu_idx]

print("WITH CTC EXPANSION:")
print(f"  Federal itemizes: {itemizes_with_ctc}")
print(f"  MA state tax: ${ma_tax_with_ctc:,.2f}")
print(f"  MA Part B exemption: ${ma_exemption_with_ctc:,.2f}")

WITH CTC EXPANSION:
  Federal itemizes: True
  MA state tax: $-4,465.42
  MA Part B exemption: $13,928.17


## Step 3: Analysis

In [4]:
# Calculate the impact
ma_tax_change = ma_tax_with_ctc - ma_tax_no_ctc
exemption_change = ma_exemption_with_ctc - ma_exemption_no_ctc

print("IMPACT OF FEDERAL CTC EXPANSION ON MA STATE TAX:")
print(f"  MA state tax reduction: ${-ma_tax_change:,.2f}")
print(f"  MA Part B exemption increase: ${exemption_change:,.2f}")

print(f"\nMECHANISM:")
print(f"1. CTC expansion triggers federal itemization: {not itemizes_no_ctc} → {itemizes_with_ctc}")
print(f"2. MA allows itemizers to claim medical expense deduction in Part B exemption")

# Show medical expense deduction
med_expense = sim_with_ctc.calculate('medical_expense_deduction', YEAR).values[tu_idx]
print(f"3. Medical expense deduction: ${med_expense:,.2f}")
print(f"4. MA tax rate: 5%")
print(f"5. Tax savings: ${exemption_change:,.2f} × 5% = ${exemption_change * 0.05:,.2f}")

# Verify federal tax is same either way
fed_tax_item = sim_with_ctc.calculate('tax_liability_if_itemizing', YEAR).values[tu_idx]
fed_tax_std = sim_with_ctc.calculate('tax_liability_if_not_itemizing', YEAR).values[tu_idx]
print(f"\nFederal tax comparison (with CTC):")
print(f"  If itemizing: ${fed_tax_item:,.2f}")
print(f"  If standard deduction: ${fed_tax_std:,.2f}")
print(f"  Difference: ${abs(fed_tax_item - fed_tax_std):,.2f}")
print(f"  → Federal tax is essentially the same either way")

IMPACT OF FEDERAL CTC EXPANSION ON MA STATE TAX:
  MA state tax reduction: $56.41
  MA Part B exemption increase: $1,128.17

MECHANISM:
1. CTC expansion triggers federal itemization: True → True
2. MA allows itemizers to claim medical expense deduction in Part B exemption
3. Medical expense deduction: $1,128.17
4. MA tax rate: 5%
5. Tax savings: $1,128.17 × 5% = $56.41

Federal tax comparison (with CTC):
  If itemizing: $-11,554.11
  If standard deduction: $-11,554.11
  Difference: $0.00
  → Federal tax is essentially the same either way
